In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
import seaborn as sns
import scipy.stats as scipy
from scipy.stats import shapiro

In [2]:
df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")

/tmp/ipykernel_283/2334064192.py:1: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")


In [3]:
df = df.dropna(subset=["VEHICLE_MAKE", 'DRIVER_LICENSE_STATUS','DRIVER_SEX', 'VEHICLE_YEAR'])
df = df[df['VEHICLE_YEAR'] <= 2025]
df = df[df['VEHICLE_OCCUPANTS'] <= 20]

In [4]:
#are Ford vehicles more prone to fatal accidents
df

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,...,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,16937807,3405925,3/22/2016,5:40,161407,NY,NaN,FORD -CAR/SUV,NaN,1997.0,...,Center Front End,NaN,NaN,NaN,N,NaN,Outside Car Distraction,Driver Inattention/Distraction,0,0
4,16946723,3406521,3/24/2016,15:30,38027,CT,NaN,BMW -CAR/SUV,NaN,2013.0,...,Left Front Bumper,NaN,NaN,NaN,N,NaN,Unsafe Speed,Following Too Closely,0,0
5,16946721,3406521,3/24/2016,15:30,38026,NY,NaN,HYUN -CAR/SUV,NaN,2013.0,...,Right Rear Bumper,Center Front End,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified,0,0
6,16947454,3406526,3/25/2016,12:20,50680,NY,Taxi,FORD -CAR/SUV,FORD ZZZ,2011.0,...,Right Front Quarter Panel,NaN,NaN,NaN,N,NaN,Passing Too Closely,NaN,0,0
7,16972534,3406722,4/15/2016,21:40,156014,NY,NaN,CHEV -CAR/SUV,NaN,2014.0,...,Right Front Quarter Panel,Right Front Bumper,NaN,NaN,N,NaN,Unsafe Lane Changing,Unspecified,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89092,17311768,3518719,9/11/2016,20:30,778066,NY,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,NaN,2016.0,...,Center Front End,Left Side Doors,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Failure to Yield Right-of-Way,11,0
89093,17414543,3578653,12/6/2016,3:00,1189706,NJ,Sedan,JEEP -CAR/SUV,NaN,2013.0,...,Center Back End,Right Rear Bumper,Left Rear Bumper,Undercarriage,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89094,17414544,3578653,12/6/2016,3:00,1189705,NJ,Sedan,HOND -CAR/SUV,NaN,2008.0,...,Center Back End,Left Rear Quarter Panel,NaN,NaN,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89095,17414542,3578653,12/6/2016,3:00,1189702,NY,Sedan,MERZ -CAR/SUV,NaN,2016.0,...,Center Front End,Left Front Bumper,Right Front Quarter Panel,Right Front Bumper,N,NaN,Alcohol Involvement,Unsafe Speed,11,0


In [5]:
df = df[['CRASH_DATE', 'CRASH_TIME', 'VEHICLE_MAKE', 'VEHICLE_YEAR','VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'DRIVER_LICENSE_STATUS', 
        'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']].copy()

In [6]:
df = df.drop(df[df['NUMBER OF PERSONS INJURED'] > df['VEHICLE_OCCUPANTS']].index)

In [7]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0
...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0


In [8]:
df['isFord'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'FORD' in x.upper() else 0)
df['Fatal'] = df['NUMBER OF PERSONS KILLED'].apply(lambda x: 1 if x > 0 else 0)
df['LicenseStatus'] = df['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Licensed' in x else 0)

In [9]:
df['Fatal'].value_counts()

Fatal
0    62517
1       36
Name: count, dtype: int64

In [10]:
df['NUMBER OF PERSONS KILLED'].value_counts()

NUMBER OF PERSONS KILLED
0    62517
1       35
3        1
Name: count, dtype: int64

In [11]:
df['DRIVER_LICENSE_STATUS'].value_counts()

DRIVER_LICENSE_STATUS
Licensed      61411
Unlicensed      820
Permit          322
Name: count, dtype: int64

In [12]:
df['VEHICLE_MAKE'].value_counts().head()

VEHICLE_MAKE
TOYT -CAR/SUV    11652
HOND -CAR/SUV     7134
NISS -CAR/SUV     6382
FORD -CAR/SUV     5725
CHEV -CAR/SUV     3146
Name: count, dtype: int64

In [13]:
df.value_counts(df['VEHICLE_MAKE'].str.contains('FORD') & df['Fatal'] == 1)

False    62545
True         8
Name: count, dtype: int64

In [14]:
df.value_counts(df['NUMBER OF PERSONS INJURED'] > 0)

NUMBER OF PERSONS INJURED
False    52547
True     10006
Name: count, dtype: int64

In [15]:
df.value_counts(df['NUMBER OF PERSONS INJURED'])

NUMBER OF PERSONS INJURED
0     52547
1      8617
2      1055
3       226
4        68
5        27
6         8
7         2
9         2
10        1
Name: count, dtype: int64

In [16]:
df.value_counts(df['VEHICLE_OCCUPANTS'] > 0)

VEHICLE_OCCUPANTS
True     59855
False     2698
Name: count, dtype: int64

In [53]:
df['VEHICLE_MAKE'].value_counts().to_csv('vehicle_counts.csv')

In [18]:
df[df['Fatal'] == 1]['VEHICLE_MAKE'].value_counts()

VEHICLE_MAKE
FORD -CAR/SUV      5
TOYT -CAR/SUV      5
CHEV -CAR/SUV      3
FORD               2
MERZ -CAR/SUV      2
GMC -CAR/SUV       2
HOND -CAR/SUV      2
SUZI -MCL          1
BMW -CAR/SUV       1
YAMA -MCL          1
MAZD -CAR/SUV      1
FORD-TRUCK/BUS     1
INTL-TRUCK/BUS     1
HYUN -CAR/SUV      1
ACUR -CAR/SUV      1
AUDI -CAR/SUV      1
VOLV -TRUCK/BUS    1
MERC -CAR/SUV      1
MACK-TRUCK/BUS     1
NE/FL              1
HOND -MCL          1
CHRY -CAR/SUV      1
Name: count, dtype: int64

In [19]:
df['Injured'] = df['NUMBER OF PERSONS INJURED'].apply(lambda x: 1 if x > 0 else 0)

In [20]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,Fatal,LicenseStatus,Injured
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,0,1,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,0,1,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,0,1,1
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,0,1,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,0,1,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,0,1,1


In [21]:
logit_y = df['Fatal']

In [22]:
#logit model
x = df['isFord']
x = sm.add_constant(x)
model1 = sm.Logit(logit_y,x).fit()

Optimization terminated successfully.
         Current function value: 0.004835
         Iterations 12


In [23]:
results_model1 = model1.summary()
results_model1

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                0.006954
Time:                        17:00:40   Log-Likelihood:                -302.44
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                   0.03958
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.6013      0.189    -40.213      0.000      -7.972      -7.231
isFord         0.9025      0.401      2.250      0.024       0.116       1.689
==============================================================================
"""

In [24]:
df['date'] = pd.to_datetime(df['CRASH_DATE'])
df['time'] = pd.to_datetime(df['CRASH_TIME'])
df['month'] = df['date'].dt.strftime('%m').astype(int)
df['day'] = df['date'].dt.strftime("%A")
df['time'] = df['time'].dt.strftime('%H%M').astype(int)
df

/tmp/ipykernel_283/3078422805.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['CRASH_TIME'])


,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,date,time,month,day,peakTime,weekday,springSummer,isHonda,isToyota,driverSex
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,2016-03-22,540,3,Tuesday,0,1,1,0,0,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,2016-03-25,1220,3,Friday,0,1,1,0,0,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,2016-04-15,2140,4,Friday,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,2016-12-01,2104,12,Thursday,0,1,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,2018-05-11,2115,5,Friday,0,1,1,0,0,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,2016-10-06,855,10,Thursday,1,1,0,0,0,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,2016-10-11,0,10,Tuesday,0,1,0,0,0,0


In [25]:
df['peakTime'] = df['time'].apply(lambda x: 1 if x >= 700 and x <= 900 else(1 if x >= 1600 and x <= 1900 else 0))
df['weekday'] = df['day'].apply(lambda x: 1 if x != 'Saturday' or x != 'Sunday' else 0)
df['springSummer'] = df['month'].apply(lambda x: 1 if x <= 8 else 0)


In [26]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,Fatal,LicenseStatus,Injured,date,time,month,day,peakTime,weekday,springSummer
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,0,1,0,2016-03-22,540,3,Tuesday,0,1,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0,2016-03-24,1530,3,Thursday,0,1,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0,2016-03-24,1530,3,Thursday,0,1,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,0,1,0,2016-03-25,1220,3,Friday,0,1,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,0,1,0,2016-04-15,2140,4,Friday,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,0,1,1,2016-12-01,2104,12,Thursday,0,1,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,0,1,1,2018-05-11,2115,5,Friday,0,1,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,0,1,1,2016-10-06,855,10,Thursday,1,1,0
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,0,1,1,2016-10-11,0,10,Tuesday,0,1,0


In [27]:
df['isHonda'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'HON' in x.upper() else 0)
df['isToyota'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'TOY' in x.upper() else 0)

In [28]:
df['driverSex'] = df['DRIVER_SEX'].apply(lambda x: 1 if x == 'M' else 0)
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,date,time,month,day,peakTime,weekday,springSummer,isHonda,isToyota,driverSex
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,2016-03-22,540,3,Tuesday,0,1,1,0,0,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,2016-03-25,1220,3,Friday,0,1,1,0,0,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,2016-04-15,2140,4,Friday,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,2016-12-01,2104,12,Thursday,0,1,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,2018-05-11,2115,5,Friday,0,1,1,0,0,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,2016-10-06,855,10,Thursday,1,1,0,0,0,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,2016-10-11,0,10,Tuesday,0,1,0,0,0,0


In [29]:
x2 = df[['isFord', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 'driverSex', 'VEHICLE_OCCUPANTS', 'springSummer']]
x2 = sm.add_constant(x2)

In [30]:
#complex logit model
model3 = sm.Logit(logit_y, x2).fit()
results_model3 = model3.summary()
results_model3

Optimization terminated successfully.
         Current function value: 0.004690
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62545
Method:                           MLE   Df Model:                            7
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.03671
Time:                        17:00:42   Log-Likelihood:                -293.38
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                  0.002201
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.8777      0.403      2.176      0.030       0.087       1.668
peakTime             -1.3567      0.531     -2.556      0.011      -2.397      -0.316
weekday               5.9598     59.180      0.101      0.920    -110.031     121.951
VEHICLE_YEAR         -0.0058      0.029     -0.196      0.845      -0.064       0.052
LicenseStatus        -1.8403      0.539     -3.412      0.001      -2.898      -0.783
driverSex            -0.0226      0.388     -0.058      0.954      -0.783       0.738
VEHICLE_OCCUPANTS     0.0284      0.159      0.179      0.858      -0.283       0.340
springSummer          0.5261      0.483      1.090      0.276      -0.420       1.472
=====================================================================================
"""

In [31]:
x3 = df[['isFord', 'isToyota', 'isHonda', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 
            'driverSex', 'VEHICLE_OCCUPANTS', 'springSummer']]
x3 = sm.add_constant(x3)


In [32]:
#logit model with other makes
model5 = sm.Logit(logit_y, x3).fit()
model5_results = model5.summary()
model5_results

Optimization terminated successfully.
         Current function value: 0.004688
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62543
Method:                           MLE   Df Model:                            9
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.03719
Time:                        17:00:44   Log-Likelihood:                -293.23
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                  0.007028
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.8097      0.420      1.926      0.054      -0.014       1.634
isToyota             -0.2089      0.503     -0.416      0.678      -1.194       0.776
isHonda              -0.2437      0.620     -0.393      0.694      -1.459       0.972
peakTime             -1.3575      0.531     -2.557      0.011      -2.398      -0.317
weekday               4.3395     59.361      0.073      0.942    -112.006     120.685
VEHICLE_YEAR         -0.0049      0.030     -0.167      0.867      -0.063       0.053
LicenseStatus        -1.8338      0.540     -3.398      0.001      -2.891      -0.776
driverSex            -0.0234      0.389     -0.060      0.952      -0.786       0.739
VEHICLE_OCCUPANTS     0.0281      0.159      0.177      0.860      -0.283       0.339
springSummer          0.5275      0.483      1.093      0.274      -0.418       1.473
=====================================================================================
"""

In [33]:
#negative binomial model
y = df['NUMBER OF PERSONS KILLED']
nb_model = sm.GLM(y, x3, family = sm.families.NegativeBinomial()).fit()
nb_results = nb_model.summary()
nb_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Generalized Linear Model Regression Results                     
====================================================================================
Dep. Variable:     NUMBER OF PERSONS KILLED   No. Observations:                62553
Model:                                  GLM   Df Residuals:                    62543
Model Family:              NegativeBinomial   Df Model:                            9
Link Function:                          Log   Scale:                          1.0000
Method:                                IRLS   Log-Likelihood:                -305.44
Date:                      Tue, 02 Dec 2025   Deviance:                       509.34
Time:                              17:00:45   Pearson chi2:                 6.25e+04
No. Iterations:                           9   Pseudo R-squ. (CS):          0.0004478
Covariance Type:                  nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                1.0149      0.389      2.606      0.009       0.252       1.778
isToyota             -0.1925      0.503     -0.383      0.702      -1.178       0.793
isHonda              -0.2411      0.620     -0.389      0.697      -1.456       0.974
peakTime             -1.4248      0.529     -2.693      0.007      -2.462      -0.388
weekday              21.9164     56.354      0.389      0.697     -88.535     132.368
VEHICLE_YEAR         -0.0138      0.028     -0.493      0.622      -0.069       0.041
LicenseStatus        -1.7236      0.538     -3.203      0.001      -2.778      -0.669
driverSex             0.0465      0.385      0.121      0.904      -0.708       0.802
VEHICLE_OCCUPANTS     0.1470      0.107      1.372      0.170      -0.063       0.357
springSummer          0.4474      0.481      0.930      0.352      -0.496       1.390
=====================================================================================
"""

In [34]:
logit_injured_y = df['Injured']
linear_injured_y = df['NUMBER OF PERSONS INJURED']

In [35]:
#injured logit model
injured_model = sm.Logit(logit_injured_y, x).fit()
injured_model_summary = injured_model.summary()
injured_model_summary

Optimization terminated successfully.
         Current function value: 0.439602
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:               3.783e-06
Time:                        17:00:46   Log-Likelihood:                -27498.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                    0.6483
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6568      0.012   -143.874      0.000      -1.679      -1.634
isFord        -0.0164      0.036     -0.455      0.649      -0.087       0.054
==============================================================================
"""

No statistical significance that ford cars cause more fatal accidents. fatal accidents are rare cannot be truly determined with OLS and Logit model

In [36]:
#injured logit model with other makes
injured_model2 = sm.Logit(logit_injured_y, x3).fit()
injured_model_summary2 = injured_model2.summary()
injured_model_summary2

Optimization terminated successfully.
         Current function value: 0.432820
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62543
Method:                           MLE   Df Model:                            9
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.01543
Time:                        17:00:47   Log-Likelihood:                -27074.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                6.690e-177
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord               -0.0139      0.037     -0.372      0.710      -0.087       0.059
isToyota              0.1235      0.029      4.269      0.000       0.067       0.180
isHonda               0.0519      0.035      1.479      0.139      -0.017       0.121
peakTime             -0.0571      0.024     -2.421      0.015      -0.103      -0.011
weekday              29.6959      3.904      7.606      0.000      22.044      37.348
VEHICLE_YEAR         -0.0155      0.002     -7.986      0.000      -0.019      -0.012
LicenseStatus        -0.4890      0.070     -6.942      0.000      -0.627      -0.351
driverSex            -0.0657      0.025     -2.664      0.008      -0.114      -0.017
VEHICLE_OCCUPANTS     0.2562      0.010     26.347      0.000       0.237       0.275
springSummer          0.1083      0.038      2.845      0.004       0.034       0.183
=====================================================================================
"""

In [37]:
#linear injured model with other makes
linear_injured_model2 = sm.OLS(linear_injured_y, x3).fit()
linear_injured_summary2 = linear_injured_model2.summary()
linear_injured_summary2

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   R-squared:                       0.047
Model:                                   OLS   Adj. R-squared:                  0.046
Method:                        Least Squares   F-statistic:                     339.4
Date:                       Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                               17:00:47   Log-Likelihood:                -42642.
No. Observations:                      62553   AIC:                         8.530e+04
Df Residuals:                          62543   BIC:                         8.539e+04
Df Model:                                  9                                         
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.0045      0.006      0.699      0.484      -0.008       0.017
isToyota              0.0231      0.005      4.521      0.000       0.013       0.033
isHonda               0.0114      0.006      1.847      0.065      -0.001       0.024
peakTime             -0.0145      0.004     -3.550      0.000      -0.022      -0.006
weekday               4.8597      0.698      6.966      0.000       3.492       6.227
VEHICLE_YEAR         -0.0024      0.000     -6.775      0.000      -0.003      -0.002
LicenseStatus        -0.0763      0.014     -5.325      0.000      -0.104      -0.048
driverSex            -0.0121      0.004     -2.798      0.005      -0.021      -0.004
VEHICLE_OCCUPANTS     0.1047      0.002     53.863      0.000       0.101       0.109
springSummer          0.0152      0.007      2.217      0.027       0.002       0.029
==============================================================================
Omnibus:                    43384.883   Durbin-Watson:                   0.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           886041.128
Skew:                           3.103   Prob(JB):                         0.00
Kurtosis:                      20.362   Cond. No.                     7.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
#negative binomial model for injured
nb2_model = sm.GLM(linear_injured_y, x3, family = sm.families.NegativeBinomial()).fit()
nb2_results = nb2_model.summary()
nb2_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   No. Observations:                62553
Model:                                   GLM   Df Residuals:                    62543
Model Family:               NegativeBinomial   Df Model:                            9
Link Function:                           Log   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -31638.
Date:                       Tue, 02 Dec 2025   Deviance:                       33774.
Time:                               17:00:49   Pearson chi2:                 5.21e+04
No. Iterations:                           16   Pseudo R-squ. (CS):            0.03394
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.0069      0.034      0.200      0.841      -0.060       0.074
isToyota              0.1176      0.027      4.376      0.000       0.065       0.170
isHonda               0.0601      0.032      1.863      0.062      -0.003       0.123
peakTime             -0.0755      0.022     -3.448      0.001      -0.118      -0.033
weekday              24.0275      3.607      6.661      0.000      16.958      31.097
VEHICLE_YEAR         -0.0129      0.002     -7.166      0.000      -0.016      -0.009
LicenseStatus        -0.3598      0.065     -5.577      0.000      -0.486      -0.233
driverSex            -0.0702      0.023     -3.103      0.002      -0.115      -0.026
VEHICLE_OCCUPANTS     0.3693      0.006     57.409      0.000       0.357       0.382
springSummer          0.0783      0.035      2.230      0.026       0.009       0.147
=====================================================================================
"""

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2ce982c-ab7e-4df7-957e-2768a79f0182' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>